In [ ]:
%%time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import xgboost as xgb

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew #for some statistics

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Concatenate, Conv1D, Activation, TimeDistributed, Flatten, RepeatVector, Permute,multiply,GlobalAveragePooling1D, GlobalMaxPooling1D ,CuDNNGRU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, GlobalAveragePooling1D, MaxPooling1D, SpatialDropout1D, BatchNormalization, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.callbacks import Callback
from keras.models import load_model

In [ ]:
from keras.callbacks import Callback
from sklearn.model_selection import KFold

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
from tqdm import tqdm_notebook
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer
import re
import string
import nltk
from nltk.corpus import stopwords

In [ ]:
tokenizer=TweetTokenizer()

In [ ]:
import os
os.listdir('../input/glove840b300dtxt')

In [ ]:
train = pd.read_csv('../input/ericsson-data/predictive_data32f5357/Predictive_Data/train_file.csv')
test = pd.read_csv('../input/ericsson-data/predictive_data32f5357/Predictive_Data/test_file.csv')
subm = pd.read_csv('../input/ericsson-data/predictive_data32f5357/Predictive_Data/sample_submission.csv')

### These having only one value so we are removing that

In [ ]:
train = train.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],axis=1)
test = test.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],axis=1)

In [ ]:
train.head()
test.head()

In [ ]:
print('class count in numbers: ')
train['MaterialType'].value_counts()
print('percentage of class count : ')
train['MaterialType'].value_counts()/train.shape[0] * 100

### encode the target vriable

In [ ]:
train['MaterialType'].replace("BOOK",0,inplace=True)
train['MaterialType'].replace("CR",1,inplace=True)
train['MaterialType'].replace("MIXED",2,inplace=True)
train['MaterialType'].replace("MUSIC",3,inplace=True)
train['MaterialType'].replace("SOUNDCASS",4,inplace=True)
train['MaterialType'].replace("SOUNDDISC",5,inplace=True)
train['MaterialType'].replace("VIDEOCASS",6,inplace=True)
train['MaterialType'].replace("VIDEODISC",7,inplace=True)

In [ ]:
## used to find out number of unique variables in each column
train.nunique()
test.nunique()

In [ ]:
## impute the nan values
train['PublicationYear'].replace(np.nan,"0000",inplace=True)
test['PublicationYear'].replace(np.nan,"0000",inplace=True)

In [ ]:
train['Subjects'].replace(np.nan,"",inplace=True)
test['Subjects'].replace(np.nan,"",inplace=True)

In [ ]:
## clean the PublicationYear column
import re
def matc(a):
    return re.findall(r"[0-9]{4}",a)
train['PublicationYear'] = train['PublicationYear'].apply(lambda x: matc(x))
test['PublicationYear'] = test['PublicationYear'].apply(lambda x: matc(x))

In [ ]:
train['Creator'].replace(np.nan,"",inplace=True)
train['Publisher'].replace(np.nan,"",inplace=True)

test['Creator'].replace(np.nan,"",inplace=True)
test['Publisher'].replace(np.nan,"",inplace=True)

### Combine all the Text columns

In [ ]:
train['combined'] = train['Title'] + ' . ' + train['Subjects'] + ' . ' + train['Creator'] + ' . ' + train['Publisher']
test['combined'] = test['Title'] + ' . ' + test['Subjects'] + ' . ' + test['Creator'] + ' . ' + test['Publisher']

In [ ]:
train['combined'].replace(np.nan,"",inplace=True)
test['combined'].replace(np.nan,"",inplace=True)

In [ ]:
def ext(a):
    #print(a)
    if len(a)==0:
        return 0
    #print(int(a[0]))
    return int(a[0])
train['PublicationYear'] = train['PublicationYear'].apply(lambda x:ext(x))
test['PublicationYear'] = test['PublicationYear'].apply(lambda x:ext(x))

### Text Cleaning before computing embeddings

In [ ]:
merge=pd.concat([train.iloc[:,8:9],test.iloc[:,8:9]])

In [ ]:
# corpus = train['combined']
corpus = merge['combined'].str.lower()
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [ ]:
"""
This function receives comments and returns clean word-list
"""
def clean(comment):
    #Convert to lower case , so that Hi and hi are the same
    #comment=comment.lower()
    #remove \n
    comment=re.sub("\\n"," ",comment)
    #removing usernames
    comment=re.sub("\[\[.*\]"," ",comment)
    #remove hyperlink
    comment=re.sub("http\S+|www.\S+"," ",comment)
    #remove numbers
    comment = clearup(comment, string.punctuation+string.digits)
    #Split the sentences into words
    words=tokenizer.tokenize(comment)
    # (')aphostophe  replacement (ie)   you're --> you are  
    # ( basic dictionary lookup : master dictionary present in a hidden block of code)
    words=[APPO[word] if word in APPO else word for word in words]
    #other commonly misspeled words.
    words=[repl[word] if word in replkeys else word for word in words]
    clean_sent=" ".join(words)
    # remove any non alphanum,digit character
    clean_sent=re.sub("\W+"," ",clean_sent)
    # remove the punctuations from the text
    remove_punctuations(clean_sent)
    clean_sent=re.sub("  "," ",clean_sent)
    clean_sent=clean_sent.strip()
    return(clean_sent)

In [ ]:
%%time
clean_corpus=corpus.apply(lambda x :clean(x))

In [ ]:
print('loading embeddings vectors')
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

### Using Glove Embeddings

In [ ]:
%%time
embeddings_index = dict(get_coefs(*o.split(' ')) for o in open('../input/glove840b300dtxt/glove.840B.300d.txt'))

In [ ]:
# list_sentences_train

In [ ]:
max_features=150000
maxlen=200

list_sentences_train = train["combined"].str.lower()
list_classes = ["MaterialType"]

train[list_classes] = train[list_classes].astype(np.int8)
target = train[list_classes]

list_sentences_test = test["combined"]
trb_nan_idx = list_sentences_test[pd.isnull(list_sentences_test)].index.tolist()
list_sentences_test.loc[trb_nan_idx] = ' '
list_sentences_test = list_sentences_test.str.lower()

print('mean text len:',train["combined"].str.count('\S+').mean())
print('max text len:',train["combined"].str.count('\S+').max())

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train) + list(list_sentences_test))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
print('padding sequences')
X_train = train["combined"]
X_test = test["combined"]
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
train[list_classes] = train[list_classes].astype(np.int8)
target = train[list_classes]

In [ ]:
print('numerical variables')
embed_size=300
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
print('create embedding matrix')
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
#embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i == 51688:
        continue
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [ ]:
# embedding_matrix[51687]
len(word_index.values())

### one hot encode the target variables

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
target = enc.fit_transform(target).toarray()

In [ ]:
target = pd.DataFrame(data=target[0:,0:], index=list(range(0,len(target))), columns=target[0,0:]) 

In [ ]:
X_train.shape
target.shape
X_test.shape

### model used is the single Gru Cnn model with concatenating the max_pool and avg_pool layer

In [ ]:
def get_single_grucnn_model():
    
    inp = Input(shape=(200, ))
    x = Embedding(51688, 300, weights=[embedding_matrix], trainable = False)(inp)
    x = SpatialDropout1D(0.3)(x)
    x2 = Bidirectional(CuDNNGRU(80, kernel_initializer='glorot_uniform', return_sequences=True))(x)
    x2 = Conv1D(120, kernel_size = 2, padding = "valid",activation = "relu",strides = 1)(x2)
    #x = Dropout(0.2)(x)
    max_pool = GlobalMaxPooling1D()(x2)
    avg_pool = GlobalAveragePooling1D()(x2)
    x = Concatenate()([avg_pool,max_pool])
    x = Dense(8, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
print('start modeling')
early_stop = EarlyStopping(monitor = "loss", mode = "min", patience = 5)
scores = []
predict = np.zeros((test.shape[0],8))
oof_predict = np.zeros((train.shape[0],8))

print("model selection")

### two fold cross-validation

In [ ]:
num_folds = 2
kf = KFold(n_splits=num_folds, shuffle=True, random_state=239)
for train_index, test_index in kf.split(X_train):
    X_train1 , X_valid = X_train[train_index] , X_train[test_index]
    y_train, y_val = target.loc[train_index], target.loc[test_index]
    model = get_single_grucnn_model()
    model.fit(X_train1, y_train, batch_size=256, epochs=10, verbose=1,callbacks = [early_stop])
    print('Predicting....')
    oof_predict[test_index] = model.predict(X_valid, batch_size=1024)
#     cv_score = roc_auc_score(y_test, oof_predict[test_index])
    #scores.append(cv_score)
    #print('score: ',cv_score)
    print('pridicting test')
    predict += model.predict(X_test, batch_size=1024)

predict = predict / num_folds
#print('Total CV score is {}'.format(np.mean(scores)))
sample_submission = pd.DataFrame.from_dict({'ID': test['ID']})

In [ ]:
preds = set(np.argmax(predict,axis=1))

### Vectorization Approach

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
train1 = tfidf_vectorizer.fit_transform(train['combined'])
test1 = tfidf_vectorizer.fit_transform(test['combined'])

In [ ]:
train['combined'].shape
test['combined'].shape

np.hstack([train['combined'].values,test['combined'].values]).shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# create the transform
vectorizer = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)
# tokenize and build vocab
vectorizer.fit(np.hstack([train['combined'].values,test['combined'].values]))
train2 = vectorizer.transform(train['combined'])
test2 = vectorizer.transform(test['combined'])
print(train2.shape)
print(type(train2))

In [ ]:
csr_matrix([train['Checkouts']]).T

### concat both the vectors together into single.

In [ ]:
from scipy.sparse import hstack, coo_matrix, csr_matrix,vstack
train3 = hstack((train1, train2))

test3 = hstack((test1, test2))

### lightGbm

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train3, train['MaterialType'], random_state=42, test_size=0.2)

# lreg = LogisticRegression()
lreg = lgb.LGBMRegressor()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict(xvalid_bow) # predicting on the validation set
# prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction.astype(np.int)

f1_score(yvalid, prediction_int,average='micro') # calculating f1 score

test_pred = lreg.predict(test3)

### Logistic

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train3, train['MaterialType'], random_state=42, test_size=0.2)

lreg = LogisticRegression()
# lreg = lgb.LGBMRegressor()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int,average='micro') # calculating f1 score

test_pred = lreg.predict_proba(test3)

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.8)
nb.fit(xtrain_bow, ytrain)

preds = nb.predict(xvalid_bow)
f1_score(yvalid, preds,average='micro') # calculating f1 score

test_pred = nb.predict(test3)

In [ ]:
subfin = pd.DataFrame({'ID': test['ID'].values, 'MaterialType': test_pred})
subfin=subfin.reindex(columns=["ID","MaterialType"])
subfin.to_csv('submission.csv', index=False)

### convert again to the submission format

In [ ]:
subfin['MaterialType'].replace(0,"BOOK",inplace=True)
subfin['MaterialType'].replace(1,"CR",inplace=True)
subfin['MaterialType'].replace(2,"MIXED",inplace=True)
subfin['MaterialType'].replace(3,"MUSIC",inplace=True)
subfin['MaterialType'].replace(4,"SOUNDCASS",inplace=True)
subfin['MaterialType'].replace(5,"SOUNDDISC",inplace=True)
subfin['MaterialType'].replace(6,"VIDEOCASS",inplace=True)
subfin['MaterialType'].replace(7,"VIDEODISC",inplace=True)

In [ ]:
subfin.to_csv('submission.csv', index=False)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(subfin)